# Creating a Data Table that contains the individual .fits file information

In [1]:
import pandas as pd 
from astropy.io import fits
import glob
import os
from pathlib import Path

In [2]:
def get_snid_folders(base_path="pantheon_data_folder"):
    """
    Returns a list of folder names (SNIDs) under the base path
    """
    return [p.name for p in Path(base_path).iterdir() if p.is_dir()]

In [3]:
def extract_fits_by_snid(snid_list, base_path="pantheon_data_folder", output_csv="fits_summary.csv", recursive=False, data_list = []):
    """
    Loops over SNIDs, reads .fits files in /pantheon/{SNID}/ folders
        - Extracts the Telescope, Instrument Used, and Filter
    Parameters:
    - snid_list: list of str, SNID folder names
    - base_path: str, base directory containing SNID subfolders (defaults to pantheon_data_folder)
    - output_csv: str, name of output CSV file
    - recursive: bool, whether to search subdirectories inside each SNID folder
    """

    for snid in snid_list:
        folder_path = os.path.join(base_path, snid)
        pattern = "**/*.fits" if recursive else "*.fits"
        file_paths = glob.glob(os.path.join(folder_path, pattern), recursive=recursive)

        for file_path in file_paths:
            try:
                with fits.open(file_path) as hdul:
                    header = hdul[0].header

                    info = {
                        "SNID": snid,
                        "filename": os.path.basename(file_path),
                        "Telescope": header.get("TELESCOP"),
                        "Instrument": header.get("INSTRUME"),
                        "Filter": header.get("FILTER"),
                    }

                    data_list.append(info)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

    # Convert to DataFrame and export
    df = pd.DataFrame(data_list)
    df.to_csv(output_csv, index=False)
    print(f"CSV exported: {output_csv}")


In [4]:
snids = get_snid_folders()
print(len(snids))
extract_fits_by_snid(snids, output_csv="fits_summary.csv", recursive=True)


38
Error reading pantheon_data_folder/2002cr/mastDownload/HST/hst_skycell-p1195x05y13_wfc3_ir_f160w_all/hst_skycell-p1195x05y13_wfc3_ir_f160w_all_drz.fits: Empty or corrupt FITS file
Error reading pantheon_data_folder/2006mr/mastDownload/HST/hst_skycell-p0497x09y14_wfc3_ir_f160w_all/hst_skycell-p0497x09y14_wfc3_ir_f160w_all_drz.fits: Empty or corrupt FITS file
CSV exported: fits_summary.csv
